In [1]:
%matplotlib widget
from util import get_path, get_dates_datetime
import pandas as pd
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from extract_graph import generate_nx_graph, transform_list, generate_skeleton, generate_nx_graph_from_skeleton, from_connection_tab, from_nx_to_tab
from node_id import whole_movement_identification, second_identification
import ast
from plotutil import plot_t_tp1, compress_skeleton
from scipy import sparse
from sparse_util import dilate, zhangSuen
from realign import realign
from datetime import datetime,timedelta
from node_id import orient
import pickle
from matplotlib.widgets import CheckButtons
import scipy.io as sio
import imageio
from pymatreader import read_mat
from matplotlib import colors
from copy import deepcopy,copy
from collections import Counter
import cv2 
import imageio
import matplotlib.pyplot as plt
import numpy as np
from skimage.filters import frangi, meijering
from skimage.morphology import thin
from skimage import data, filters
from random import choice
import scipy.sparse
import os
from time import time
from extract_graph import dic_to_sparse, from_sparse_to_graph, generate_nx_graph, prune_graph, from_nx_to_tab, from_nx_to_tab_matlab,sparse_to_doc, connections_pixel_list_to_tab, transform_list, clean_degree_4
from time import sleep
from skimage.feature import hessian_matrix_det
from experiment_class_surf import Experiment,clean_exp_with_hyphaes
from hyphae_id_surf import clean_and_relabel, get_mother, save_hyphaes, resolve_ambiguity_two_ends,solve_degree4, clean_obvious_fake_tips


In [2]:
plate = 3
directory = "/scratch/shared/mrozemul/Fiji.app/" 
listdir=os.listdir(directory) 
list_dir_interest=[name for name in listdir if name.split('_')[-1]==f'Plate{0 if plate<10 else ""}{plate}']
len(list_dir_interest)

20

In [3]:
dates_datetime = get_dates_datetime(directory,plate)
begin = 0
end = 19
print('begin =',dates_datetime[begin],'\n  end =',dates_datetime[end])

begin = 2020-08-14 00:57:00 
  end = 2020-08-17 04:57:00


In [4]:
#chose i equal to the timestep where something wrong has been noticed 
i = 0

dates_datetime = get_dates_datetime(directory,plate)
dates_datetime_chosen=dates_datetime
dates = [f'{0 if date.month<10 else ""}{date.month}{0 if date.day<10 else ""}{date.day}_{0 if date.hour<10 else ""}{date.hour}{0 if date.minute<10 else ""}{date.minute}' for date in dates_datetime_chosen]
date =dates [i]
directory_name=f'2020{date}_Plate{0 if plate<10 else ""}{plate}'
path_snap=directory+directory_name
path_tile=path_snap+'/Img/TileConfiguration.txt.registered'
try:
    tileconfig = pd.read_table(path_tile,sep=';',skiprows=4,header=None,converters={2 : ast.literal_eval},skipinitialspace=True)
except:
    print('error_name')
    path_tile=path_snap+'/Img/TileConfiguration.registered.txt'
    tileconfig = pd.read_table(path_tile,sep=';',skiprows=4,header=None,converters={2 : ast.literal_eval},skipinitialspace=True)
t=time()
xs =[c[0] for c in tileconfig[2]]
ys =[c[1] for c in tileconfig[2]]
dim = (int(np.max(ys)-np.min(ys))+4096,int(np.max(xs)-np.min(xs))+4096)
ims = []
for name in tileconfig[0]:
#     ims.append(imageio.imread('//sun.amolf.nl/shimizu-data/home-folder/oyartegalvez/Drive_AMFtopology/PRINCE'+date_plate+plate_str+'/Img/'+name))
    ims.append(imageio.imread(f'{name}'))

In [111]:
plt.close('all')

#chose a spot where to look closer at
linex  = 3000
liney = 1500

shape = (3000,4096)
linex *= 5
liney *= 5
images = []
for index,im in enumerate(ims):
    boundaries = int(tileconfig[2][index][0]-np.min(xs)),int(tileconfig[2][index][1]-np.min(ys))
    if boundaries[1]<=linex< boundaries[1]+shape[0] and boundaries[0]<=liney< boundaries[0]+shape[1]:
        images.append(im)


In [112]:
plt.close('all')
im = images[0]
fig=plt.figure(figsize=(10,9))
ax = fig.add_subplot(111)
ax.imshow(im)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [113]:
plt.close('all')
im = images[0]
im_cropped = im
im_blurred =cv2.blur(im_cropped, (200, 200))
im_back_rem = (im_cropped+1)/(im_blurred+1)*120
# # im_back_rem = im_cropped*1.0
# # # im_back_rem = cv2.normalize(im_back_rem, None, 0, 255, cv2.NORM_MINMAX)
frangised = frangi(im_back_rem,sigmas=range(1,20,4))*255
# # frangised = cv2.normalize(frangised, None, 0, 255, cv2.NORM_MINMAX)
hessian = hessian_matrix_det(im_back_rem,sigma = 20)
blur_hessian = cv2.blur(abs(hessian), (20, 20))
#     transformed = (frangised+cv2.normalize(blur_hessian, None, 0, 255, cv2.NORM_MINMAX)-im_back_rem+120)*(im_blurred>=35)
#     transformed = (frangised+cv2.normalize(abs(hessian), None, 0, 255, cv2.NORM_MINMAX)-im_back_rem+120)*(im_blurred>=35)
transformed = (frangised-im_back_rem+120)*(im_blurred>=35)
low = 20
high = 100
lowt = (transformed > low).astype(int)
hight = (transformed > high).astype(int)
hyst = filters.apply_hysteresis_threshold(transformed, low, high)
kernel = np.ones((3,3),np.uint8)
dilation = cv2.dilate(hyst.astype(np.uint8) * 255,kernel,iterations = 1)
for i in range(3):
    dilation=cv2.erode(dilation.astype(np.uint8) * 255,kernel,iterations = 1)
    dilation = cv2.dilate(dilation.astype(np.uint8) * 255,kernel,iterations = 1)
dilated = dilation>0
skeletonized = cv2.ximgproc.thinning(np.array(255*dilated,dtype=np.uint8))

In [114]:
plot_t_tp1([],[],None,None,dilated,skeletonized)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [115]:
nx_g = generate_nx_graph(from_sparse_to_graph(scipy.sparse.dok_matrix(skeletonized)))
g,pos= nx_g
tips = [node for node in g.nodes if g.degree(node)==1]
dilated_bis = np.copy(dilated)
for tip in tips:
    branch = np.array(orient(g.get_edge_data(*list(g.edges(tip))[0])['pixel_list'],pos[tip]))
    orientation = branch[0]-branch[min(branch.shape[0]-1,20)]
    orientation = orientation/(np.linalg.norm(orientation))
    window = 20
    x,y = pos[tip][0],pos[tip][1]
    if x-window>=0 and x+window< dilated.shape[0] and y-window>=0 and y+window< dilated.shape[1]:
        shape = dilated[x-window:x+window,y-window:y+window]
        dist = 30
        for i in range(dist):
            pixel = (pos[tip]+orientation*i).astype(int)
            xp,yp = pixel[0],pixel[1]
            if xp-window>=0 and xp+window< dilated.shape[0] and yp-window>=0 and yp+window< dilated.shape[1]:
                dilated_bis[xp-window:xp+window,yp-window:yp+window]+=shape
skeletonized_bis = cv2.ximgproc.thinning(np.array(255*dilated_bis,dtype=np.uint8))

In [116]:
plot_t_tp1([],[],None,None,cv2.dilate(skeletonized_bis.astype(np.uint8) * 255,kernel,iterations = 1),im_back_rem)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [110]:
fig=plt.figure(figsize=(10,9))
ax = fig.add_subplot(111)
ax.imshow(transformed)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [38]:
orientation = branch[0]-branch[min(branch.shape[0]-1,20)]

In [ ]:
branch

In [26]:
extension = branch[0:min(branch.shape[0],20)] + orientation

In [ ]:
for edge in g.edges:
    

In [92]:
extension

array([[   0, 1610],
       [   0, 1611],
       [   0, 1612],
       [   0, 1613],
       [   0, 1614],
       [   0, 1615],
       [   0, 1616],
       [   0, 1617],
       [   0, 1618],
       [   0, 1619],
       [   0, 1620],
       [   0, 1621],
       [   0, 1622],
       [   0, 1623],
       [   0, 1624],
       [   0, 1625],
       [   0, 1626],
       [   0, 1627],
       [   0, 1628],
       [   0, 1629]])

In [42]:
plt.close('all')
im = images[0]
im_cropped = im
im_blurred =cv2.blur(im_cropped, (200, 200))
im_back_rem = (im_cropped+1)/(im_blurred+1)*120
test=np.copy(im_back_rem)
# test = test * (test<128)+(256-test)*(test>=128)
# test =cv2.blur(test, (20, 20))
test[test >= 130] = 130
fig=plt.figure(figsize=(10,9))
ax = fig.add_subplot(111)
ax.imshow(im_back_rem)
fig=plt.figure(figsize=(10,9))
ax = fig.add_subplot(111)
ax.imshow(test)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [43]:
frangised = frangi(test,sigmas=range(1,20,4))*255


In [44]:
fig=plt.figure(figsize=(10,9))
ax = fig.add_subplot(111)
ax.imshow(frangised)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [45]:
frangi_blurred =cv2.blur(frangised,(20,20))
fig=plt.figure(figsize=(10,9))
ax = fig.add_subplot(111)
ax.imshow((frangi_blurred-im_back_rem+120)*(im_blurred>=35))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [46]:
plt.close('all')
transformed = (frangi_blurred-im_back_rem+120)*(im_blurred>=35)
low = 10
high = 30
lowt = (transformed > low).astype(int)
hight = (transformed > high).astype(int)
hyst = filters.apply_hysteresis_threshold(transformed, low, high)
kernel = np.ones((3,3),np.uint8)
dilation = cv2.dilate(hyst.astype(np.uint8) * 255,kernel,iterations = 1)
for i in range(3):
    dilation=cv2.erode(dilation.astype(np.uint8) * 255,kernel,iterations = 1)
    dilation = cv2.dilate(dilation.astype(np.uint8) * 255,kernel,iterations = 1)
dilated = dilation>0
hessian = hessian_matrix_det(dilated,sigma = 15)
hessian = cv2.normalize(-hessian, None, 0, 255, cv2.NORM_MINMAX)


In [51]:
plt.close('all')
fig=plt.figure(figsize=(10,9))
ax = fig.add_subplot(111)
ax.imshow(hessian)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [47]:
kernel = np.ones((20,20),np.uint8)
junction_connecter = hessian>165
junction_connecter =  cv2.dilate(junction_connecter.astype(np.uint8) * 255,kernel,iterations = 1)
final = dilated-junction_connecter>0
kernel = np.ones((5,5),np.uint8)
sum_eroded=cv2.erode(final.astype(np.uint8) * 255,kernel,iterations = 1)
print('Raw image')
#         fig=plt.figure(figsize=(10,9))
#         ax = fig.add_subplot(111)
#         ax.imshow(im)
print('Raw image with background removed')
fig=plt.figure(figsize=(10,9))
ax = fig.add_subplot(111)
ax.imshow(im_back_rem)
#         print('frangised image')
#         fig=plt.figure(figsize=(10,9))
#         ax = fig.add_subplot(111)
#         ax.imshow(frangised)
#         print('final transformed image')
# fig=plt.figure(figsize=(10,9))
# ax = fig.add_subplot(111)
# ax.imshow(transformed)
#         print('threhsolded image')
fig=plt.figure(figsize=(10,9))
ax = fig.add_subplot(111)
ax.imshow(sum_eroded)
#         fig=plt.figure(figsize=(10,9))
#         ax = fig.add_subplot(111)
#         ax.imshow(cv2.normalize(abs(hessian), None, 0, 255, cv2.NORM_MINMAX)-255*dilated)

Raw image
Raw image with background removed


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …